In [1]:
import torch  # This is all you need to use both PyTorch and TorchScript!
print(torch.__version__)

1.8.1+cu101


In [2]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()

    def forward(self, x, h):
        new_h = torch.tanh(x + h)
        return new_h, new_h

my_cell = MyCell()
x = torch.rand(3, 4)
h = torch.rand(3, 4)
print(my_cell(x, h))

(tensor([[0.9017, 0.8036, 0.8613, 0.6970],
        [0.8208, 0.4421, 0.2802, 0.8509],
        [0.9452, 0.7254, 0.8021, 0.9145]]), tensor([[0.9017, 0.8036, 0.8613, 0.6970],
        [0.8208, 0.4421, 0.2802, 0.8509],
        [0.9452, 0.7254, 0.8021, 0.9145]]))


In [3]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return new_h, new_h

my_cell = MyCell()
print(my_cell)
print(my_cell(x, h))

MyCell(
  (linear): Linear(in_features=4, out_features=4, bias=True)
)
(tensor([[-0.0124,  0.6679,  0.5074,  0.7585],
        [-0.6123,  0.5883, -0.2214,  0.6873],
        [-0.3758,  0.5681,  0.5151,  0.9080]], grad_fn=<TanhBackward>), tensor([[-0.0124,  0.6679,  0.5074,  0.7585],
        [-0.6123,  0.5883, -0.2214,  0.6873],
        [-0.3758,  0.5681,  0.5151,  0.9080]], grad_fn=<TanhBackward>))


In [4]:
class MyDecisionGate(torch.nn.Module):
    def forward(self, x):
        if x.sum() > 0:
            return x
        else:
            return -x

class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.dg = MyDecisionGate()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h, new_h

my_cell = MyCell()
print(my_cell)
print(my_cell(x, h))

MyCell(
  (dg): MyDecisionGate()
  (linear): Linear(in_features=4, out_features=4, bias=True)
)
(tensor([[0.8738, 0.7788, 0.6384, 0.4429],
        [0.4648, 0.7320, 0.2637, 0.0326],
        [0.7194, 0.7710, 0.7433, 0.5353]], grad_fn=<TanhBackward>), tensor([[0.8738, 0.7788, 0.6384, 0.4429],
        [0.4648, 0.7320, 0.2637, 0.0326],
        [0.7194, 0.7710, 0.7433, 0.5353]], grad_fn=<TanhBackward>))


In [5]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return new_h, new_h

my_cell = MyCell()
x, h = torch.rand(3, 4), torch.rand(3, 4)
traced_cell = torch.jit.trace(my_cell, (x, h))
print(traced_cell)
traced_cell(x, h)

MyCell(
  original_name=MyCell
  (linear): Linear(original_name=Linear)
)


(tensor([[ 0.7270,  0.1143,  0.8299,  0.6226],
         [ 0.2723,  0.4506,  0.6377, -0.0657],
         [ 0.7480, -0.1361,  0.2966,  0.1647]], grad_fn=<TanhBackward>),
 tensor([[ 0.7270,  0.1143,  0.8299,  0.6226],
         [ 0.2723,  0.4506,  0.6377, -0.0657],
         [ 0.7480, -0.1361,  0.2966,  0.1647]], grad_fn=<TanhBackward>))

In [6]:
print(traced_cell.graph)

graph(%self.1 : __torch__.MyCell,
      %input : Float(3, 4, strides=[4, 1], requires_grad=0, device=cpu),
      %h : Float(3, 4, strides=[4, 1], requires_grad=0, device=cpu)):
  %21 : __torch__.torch.nn.modules.linear.Linear = prim::GetAttr[name="linear"](%self.1)
  %23 : Tensor = prim::CallMethod[name="forward"](%21, %input)
  %14 : int = prim::Constant[value=1]() # <ipython-input-5-1f6e08af67d0>:7:0
  %15 : Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu) = aten::add(%23, %h, %14) # <ipython-input-5-1f6e08af67d0>:7:0
  %16 : Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu) = aten::tanh(%15) # <ipython-input-5-1f6e08af67d0>:7:0
  %17 : (Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu), Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu)) = prim::TupleConstruct(%16, %16)
  return (%17)



In [7]:
print(traced_cell.code)

def forward(self,
    input: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  _0 = torch.add((self.linear).forward(input, ), h, alpha=1)
  _1 = torch.tanh(_0)
  return (_1, _1)



In [8]:
print(my_cell(x, h))
print(traced_cell(x, h))

(tensor([[ 0.7270,  0.1143,  0.8299,  0.6226],
        [ 0.2723,  0.4506,  0.6377, -0.0657],
        [ 0.7480, -0.1361,  0.2966,  0.1647]], grad_fn=<TanhBackward>), tensor([[ 0.7270,  0.1143,  0.8299,  0.6226],
        [ 0.2723,  0.4506,  0.6377, -0.0657],
        [ 0.7480, -0.1361,  0.2966,  0.1647]], grad_fn=<TanhBackward>))
(tensor([[ 0.7270,  0.1143,  0.8299,  0.6226],
        [ 0.2723,  0.4506,  0.6377, -0.0657],
        [ 0.7480, -0.1361,  0.2966,  0.1647]], grad_fn=<TanhBackward>), tensor([[ 0.7270,  0.1143,  0.8299,  0.6226],
        [ 0.2723,  0.4506,  0.6377, -0.0657],
        [ 0.7480, -0.1361,  0.2966,  0.1647]], grad_fn=<TanhBackward>))


In [9]:
class MyDecisionGate(torch.nn.Module):
    def forward(self, x):
        if x.sum() > 0:
            return x
        else:
            return -x

class MyCell(torch.nn.Module):
    def __init__(self, dg):
        super(MyCell, self).__init__()
        self.dg = dg
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h, new_h

my_cell = MyCell(MyDecisionGate())
traced_cell = torch.jit.trace(my_cell, (x, h))

print(traced_cell.dg.code)
print(traced_cell.code)

def forward(self,
    argument_1: Tensor) -> Tensor:
  return torch.neg(argument_1)

def forward(self,
    input: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  _0 = (self.dg).forward((self.linear).forward(input, ), )
  _1 = torch.tanh(torch.add(_0, h, alpha=1))
  return (_1, _1)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
scripted_gate = torch.jit.script(MyDecisionGate())

my_cell = MyCell(scripted_gate)
scripted_cell = torch.jit.script(my_cell)

print(scripted_gate.code)
print(scripted_cell.code)

def forward(self,
    x: Tensor) -> Tensor:
  _0 = bool(torch.gt(torch.sum(x, dtype=None), 0))
  if _0:
    _1 = x
  else:
    _1 = torch.neg(x)
  return _1

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  _0 = (self.dg).forward((self.linear).forward(x, ), )
  new_h = torch.tanh(torch.add(_0, h, alpha=1))
  return (new_h, new_h)



In [11]:
# New inputs
x, h = torch.rand(3, 4), torch.rand(3, 4)
traced_cell(x, h)

(tensor([[0.3651, 0.5820, 0.7911, 0.3174],
         [0.3868, 0.7324, 0.8736, 0.0433],
         [0.1338, 0.0247, 0.5702, 0.4735]], grad_fn=<TanhBackward>),
 tensor([[0.3651, 0.5820, 0.7911, 0.3174],
         [0.3868, 0.7324, 0.8736, 0.0433],
         [0.1338, 0.0247, 0.5702, 0.4735]], grad_fn=<TanhBackward>))

In [12]:
class MyRNNLoop(torch.nn.Module):
    def __init__(self):
        super(MyRNNLoop, self).__init__()
        self.cell = torch.jit.trace(MyCell(scripted_gate), (x, h))

    def forward(self, xs):
        h, y = torch.zeros(3, 4), torch.zeros(3, 4)
        for i in range(xs.size(0)):
            y, h = self.cell(xs[i], h)
        return y, h

rnn_loop = torch.jit.script(MyRNNLoop())
print(rnn_loop.code)

def forward(self,
    xs: Tensor) -> Tuple[Tensor, Tensor]:
  h = torch.zeros([3, 4], dtype=None, layout=None, device=None, pin_memory=None)
  y = torch.zeros([3, 4], dtype=None, layout=None, device=None, pin_memory=None)
  y0 = y
  h0 = h
  for i in range(torch.size(xs, 0)):
    _0 = (self.cell).forward(torch.select(xs, 0, i), h0, )
    y1, h1, = _0
    y0, h0 = y1, h1
  return (y0, h0)



In [13]:
class WrapRNN(torch.nn.Module):
    def __init__(self):
        super(WrapRNN, self).__init__()
        self.loop = torch.jit.script(MyRNNLoop())

    def forward(self, xs):
        y, h = self.loop(xs)
        return torch.relu(y)

traced = torch.jit.trace(WrapRNN(), (torch.rand(10, 3, 4)))
print(traced.code)

def forward(self,
    argument_1: Tensor) -> Tensor:
  _0, y, = (self.loop).forward(argument_1, )
  return torch.relu(y)



In [14]:
traced.save('wrapped_rnn.pt')

loaded = torch.jit.load('wrapped_rnn.pt')

print(loaded)
print(loaded.code)

RecursiveScriptModule(
  original_name=WrapRNN
  (loop): RecursiveScriptModule(
    original_name=MyRNNLoop
    (cell): RecursiveScriptModule(
      original_name=MyCell
      (dg): RecursiveScriptModule(original_name=MyDecisionGate)
      (linear): RecursiveScriptModule(original_name=Linear)
    )
  )
)
def forward(self,
    argument_1: Tensor) -> Tensor:
  _0, y, = (self.loop).forward(argument_1, )
  return torch.relu(y)

